In [53]:
# Load the usual suspects
import csv
import pandas as pd
import numpy as np
import os
import Toolbox as tb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Initialize warning log container 
log = list()

In [ ]:
# Load Pre-processed data from Data Processing PCA.ipynb
FM_data = pd.read_csv(os.path.dirname(os.getcwd()) + '\\FM_data.csv')
returns = pd.read_csv(os.path.dirname(os.getcwd()) + '\\returns_data.csv').set_index(["permno", "date"])
industry_code = pd.read_csv(os.path.dirname(os.getcwd()) + '\\industry_codes.csv').set_index(["permno", "date"])

tb.downcast(FM_data)
tb.downcast(returns)
tb.downcast(industry_code)

In [49]:
# Dates partitioning the training, validation, and test set according to Gu, Kelly, and Xiu (2020) by rolling-windows approach
tb.partition_dates(iteration = 0) # 30 splits --> iteration = [0;29]

(195701, 197501, 197501, 198701, 198701, 198801)

In [54]:
tr_firm, v_firm, t_firm, tr_macro, v_macro, t_macro = tb.data_processing_new(data = FM_data, iteration = 0)

In [ ]:
# interaction tager data og iteration (så data_processing og partition_dates virker) herefter laver den interaction terms for training, validation og test sæt og 
# spytter så dem tilbage; tr_interaction, v_interaction, t_interaction. Jeg skal huske at test og validation skal bruge mean og std. fra træning! 

# Den skal derfor nok lave træning først og så bruge dens mean og std til at lave vali og test 

# Kan lave nyt .py dokument med support function, som er de funktioner jeg bruger i de store funktioner i toolbox.py for at gøre det mere clean. Kan også overveje at lave toolbox2.py i stedet

In [ ]:
def interaction_terms(firm_data, macro_data, mean, std, training_set = False):
    
    # Initialize empty dataframe
    interaction = pd.DataFrame(columns = range(0), index = range(firm_data.shape[0]))
    
    # Computer interaction terms
    for count, value in enumerate(macro_data.columns):
        
        macro_ite = macro_data[value].values # .values returns np.array not pd.series
        product_ite = macro_ite.reshape(-1,1) * firm_data.values # 2D to make compatible for element-wise multiplication       
        column_ite = [str(col) + f'X{value}' for col in firm_data.columns] 
        df_ite = pd.DataFrame(product_ite, columns = column_ite)
        interaction = pd.concat([interaction, df_ite], axis = 1)
        
    
    # Save mean and standard deviation for validation and test set
    # Impute data and standardize training data in preperation
    # of PCA
    if training_set:
        mean = interaction.mean().values.reshape(-1,1).T
        std = interaction.std().values
        
        interaction = interaction.fillna(0)
        interaction.apply(lambda x: (x - np.mean(x) / np.std(x), axis = 0)
                
        return interaction, mean, std
                    
    interaction = interaction.fillna(0)
    interaction.apply(lambda x: (x - mean / std, axis = 1)                      
    #interaction = standardized(interaction, mean, std)
                          
    return interaction
        
    
        

    
    

In [ ]:
def interaction(data, iteration):
    
    # Load training, validation, and test data 
    # for firm and macro data
    tr_firm, v_firm, t_firm, tr_macro, v_macro, t_macro = data_processing_new(data, iteration)
    
    a
    
    # Initialize empty dataframe
    interaction = pd.DataFrame(columns = range(0), index = range(firm_data.shape[0]))
    
    for count, value in enumerate(macro_data.columns):
                          
        macro_ite = macro_data[value].values # .values returns np.array not pd.series
        product_ite = macro_ite.reshape(-1,1) * firm_data.values # 2D to make compatible for element-wise multiplication       
        column_ite = [str(col) + f'X{value}' for col in firm_data.columns] 
        df_ite = pd.DataFrame(product_ite, columns = column_ite)
        interaction = pd.concat([interaction, df_ite], axis = 1)
        

    # Save mean and standardization for validation and test set 
    mean = interaction.mean().values.reshape(-1,1).T
    std = interaction.std().values
    
    # standardize and imputate data in preperation of PCA
    interaction = interaction.apply(lambda x: (x - np.mean(x)) / np.std(x), axis = 0)
    interaction = interaction.fillna(0)
    
    return interaction, mean, std

In [85]:
FM_data.head()

,permno,date,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,zerotrade,dp_macro,ep_macro,bm_macro,ntis,tbl,tms,dfy,svar,constant
0,10006,195701,82249.000,1.122846,1.260784,0.047180,9.569954,0.025742,0.046433,0.044843,...,1.120996e-07,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
1,10014,195701,3903.375,0.426734,0.182102,-0.275641,6.237836,0.072103,0.046433,-0.086957,...,9.229146e-08,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
2,10022,195701,9273.250,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046433,-0.060377,...,1.181757e-07,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
3,10030,195701,54465.875,0.926038,0.857547,0.018171,9.825336,0.021700,0.046433,0.044633,...,6.126699e-08,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1
4,10057,195701,40250.000,1.247748,1.556875,0.025785,7.901007,0.025506,0.046433,0.086667,...,3.315790e+00,3.248434,2.574677,0.567243,0.027992,0.0311,0.0017,0.0072,0.000902,1


In [95]:
FM_mean = FM_data.head().mean()
FM_std = FM_data.head().std()
FM_mean

permno       10025.800000
date        195701.000000
mvel1        38028.300781
beta             0.957963
betasq           0.998924
                ...      
tbl              0.031100
tms              0.001700
dfy              0.007200
svar             0.000902
constant         1.000000
Length: 105, dtype: float64

In [96]:
FM_data.head().apply(lambda x: (x - FM_mean) / FM_std, axis = 1)

,permno,date,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,zerotrade,dp_macro,ep_macro,bm_macro,ntis,tbl,tms,dfy,svar,constant
0,-1.010151,NaN,1.361126,0.517442,0.502505,0.669225,0.931962,-0.416802,NaN,0.520449,...,-0.447214,NaN,NaN,NaN,0.894427,NaN,-0.894427,NaN,NaN,NaN
1,-0.602009,NaN,-1.050375,-1.667117,-1.567470,-1.753327,-1.193053,1.779487,NaN,-1.234736,...,-0.447214,NaN,NaN,NaN,0.894427,NaN,-0.894427,NaN,NaN,NaN
2,-0.193867,NaN,-0.885089,0.340453,0.265566,0.123891,-0.701352,-0.326488,NaN,-0.880779,...,-0.447214,NaN,NaN,NaN,0.894427,NaN,-0.894427,NaN,NaN,NaN
3,0.214275,NaN,0.505953,-0.100187,-0.271301,0.451537,1.094829,-0.608248,NaN,0.517649,...,-0.447214,NaN,NaN,NaN,0.894427,NaN,-0.894427,NaN,NaN,NaN
4,1.591754,NaN,0.068385,0.909411,1.070699,0.508673,-0.132387,-0.427949,NaN,1.077417,...,1.788854,NaN,NaN,NaN,0.894427,NaN,-0.894427,NaN,NaN,NaN


In [115]:
(FM_data.head().beta - FM_data.head().beta.mean()) / FM_data.head().beta.std()

0    0.517442
1   -1.667117
2    0.340453
3   -0.100187
4    0.909411
Name: beta, dtype: float32

In [92]:
FM_data.head().indmom.std()

0.0

In [108]:
#nunique() == 1

column_test = [column for column in FM_data.head().columns if (pd.Series.nunique(FM_data.head()[column]) == 1)]
#drop_zero = [column for column in data.columns if (data[column] == 0).all()]